In [1]:
# General imports
import numpy as np
import torch

from deepymod.data import Dataset
from deepymod.data.burgers import BurgersDelta
from sklearn.linear_model import ARDRegression

import seaborn as sns
sns.set()

# Making data

In [2]:
# Making dataset
v = 0.1
A = 1.0

x = np.linspace(-3, 4, 100)
t = np.linspace(0.5, 5.0, 50)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(BurgersDelta, v=v, A=A)

y = dataset.time_deriv(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1)) # observations
X = dataset.library(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), poly_order=2, deriv_order=3) # covariates

print(y.shape, X.shape)

(5000, 1) (5000, 12)


In [3]:
y += np.std(y) * 0.5 * np.random.randn(*y.shape)

In [4]:
np.std(y) * 0.5

0.08978544604764829

In [5]:
X = X / np.linalg.norm(X, axis=0, keepdims=True)

# Own implementation

In [99]:
def robust_SBL(X, y, alpha_, beta_, lambda_):
    N, M = X.shape[0], X.shape[1]
    
    gram = X.T @ X
    sigma = torch.inverse(torch.diag(1/(beta_* alpha_)) + gram)
    mn = sigma @ X.T @ y
    
    p = - N * torch.log(beta_) - torch.logdet(torch.eye(M) -  gram @ sigma) + beta_ * (y.T @ y - y.T @ X @ mn) - 2 * (N-1) * 
    return p, mn

In [118]:
# Now let's optimize
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

a = torch.nn.Parameter(-1 * torch.ones(12, dtype=torch.float32))
b = torch.nn.Parameter(-torch.log(torch.var(y)))
l = torch.nn.Parameter(-1 * torch.ones(1, dtype=torch.float32))

optimizer = torch.optim.Adam([a, b, l], lr=1e-3)
max_epochs=1e4

<ipython-input-118-197330582e2d>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)
<ipython-input-118-197330582e2d>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float32)


In [141]:
for epoch in torch.arange(max_epochs):
    alpha_ = torch.exp(a).clamp(min=1e-8, max=1e8)
    beta_ = torch.exp(b).clamp(min=1e-8, max=1e8)
    lambda_ = torch.exp(l).clamp(min=1e-8, max=1e8)
    loss = robust_SBL(X, y, alpha_, beta_, lambda_)[0]
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(loss)

tensor([[-170041.7188]], grad_fn=<AddBackward0>)
tensor([[-180064.0156]], grad_fn=<AddBackward0>)
tensor([[-190073.8594]], grad_fn=<AddBackward0>)
tensor([[-196048.7188]], grad_fn=<AddBackward0>)
tensor([[-196181.2969]], grad_fn=<AddBackward0>)
tensor([[-196242.0156]], grad_fn=<AddBackward0>)
tensor([[-196275.7656]], grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [142]:
alpha_

tensor([1.0000e-08, 1.3109e-07, 2.0758e-07, 1.0000e-08, 1.0000e-08, 1.9607e-07,
        1.0000e-08, 2.8810e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08],
       grad_fn=<ClampBackward>)

In [143]:
mn = robust_SBL(X, y, alpha_, beta_, lambda_)[1]
print(mn)

tensor([[ 2.6569e-08],
        [-3.7538e-05],
        [ 3.0696e-05],
        [ 2.1581e-06],
        [-8.3937e-07],
        [-5.3635e-05],
        [ 8.3212e-07],
        [ 6.7567e-06],
        [-1.2921e-06],
        [-2.3278e-06],
        [ 6.6815e-07],
        [ 2.1088e-06]], grad_fn=<MmBackward>)


In [144]:
mn

tensor([[ 2.6569e-08],
        [-3.7538e-05],
        [ 3.0696e-05],
        [ 2.1581e-06],
        [-8.3937e-07],
        [-5.3635e-05],
        [ 8.3212e-07],
        [ 6.7567e-06],
        [-1.2921e-06],
        [-2.3278e-06],
        [ 6.6815e-07],
        [ 2.1088e-06]], grad_fn=<MmBackward>)

In [146]:
(y.T @ y - y.T @ X @ mn)

tensor([[161.2351]], grad_fn=<SubBackward0>)

In [130]:
lambda_

tensor([74966.4766], grad_fn=<ClampBackward>)

In [131]:
1 / beta_

tensor(0.0142, grad_fn=<MulBackward0>)

In [135]:
1 / alpha_ < 1e4

tensor([False,  True,  True, False, False,  True, False,  True, False, False,
        False, False])

In [75]:
N, M = X.shape[0], X.shape[1]

gram = X.T @ X
sigma = torch.inverse(torch.diag(1/(beta_* alpha_)) + gram)
mn = sigma @ X.T @ y

p = - N * torch.log(beta_) - torch.logdet(torch.eye(M) -  gram @ sigma) + beta_ * (y.T @ y - y.T @ X @ mn) #- 2 * (N-1) * torch.log(lambda_) + torch.exp(torch.log(lambda_) + torch.log(torch.sum(alpha_)))

In [76]:
print(p)

tensor([[-20067.1641]], grad_fn=<AddBackward0>)


In [77]:
- N * torch.log(beta_)

tensor(-25088.7578, grad_fn=<MulBackward0>)

In [78]:
- torch.logdet(torch.eye(M) -  gram @ sigma)

tensor(19.0785, grad_fn=<NegBackward>)

In [79]:
 beta_ * (y.T @ y - y.T @ X @ mn)

tensor([[5002.5161]], grad_fn=<MulBackward0>)

In [80]:
- 2 * (N-1) * torch.log(lambda_)

tensor([99980.], grad_fn=<MulBackward0>)

In [81]:
lambda_ * torch.sum(alpha_)

tensor([0.0075], grad_fn=<MulBackward0>)

In [84]:
N / torch.sum(alpha_/2)

tensor(60.4984, grad_fn=<MulBackward0>)